In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
ciclovias_geo = '../data bicicletas/referencia_geografica_ciclovias_WGS84.geojson'
puntos_censos_anuales = '../data bicicletas/puntos-censos-anuales.csv'
puntos_censos_mensuales = '../data bicicletas/puntos-censos-mensuales.csv'
volumen_ciclista_anual = '../data bicicletas/volumen-ciclistas-anuales.csv'
volumen_ciclista_mensual = '../data bicicletas/volumen-ciclistas-mensuales.csv'

### Normalizamos CSVs para luego exportar antes de cargar a SQL server

In [3]:
df_ciclovias = gpd.read_file(ciclovias_geo)
df_puntos_censos_anuales = pd.read_csv(puntos_censos_anuales)
df_puntos_censos_mensuales = pd.read_csv(puntos_censos_mensuales)
df_volumen_ciclista_anual = pd.read_csv(volumen_ciclista_anual)
df_volumen_ciclista_mensual = pd.read_csv(volumen_ciclista_mensual)

ciclovias

In [11]:
ciclovias_normalizada = df_ciclovias
ciclovias_normalizada.rename(columns={'ciclo_obse': 'anio_construccion'}, inplace=True)
ciclovias_normalizada['anio_construccion'] = ciclovias_normalizada['anio_construccion'].str.replace(r'\D', '').replace('',np.nan)

for x in ciclovias_normalizada['anio_construccion']:
    if len(str(x)) != 4:
        ciclovias_normalizada['anio_construccion'].replace(x, np.nan, inplace=True)

# ciclovias_normalizada['anio_construccion'] = pd.to_datetime(ciclovias_normalizada['anio_construccion'], format='%Y', errors='coerce')
# ciclovias_normalizada['anio_construccion'].replace({np.nan: None}, inplace=True)
# ciclovias_normalizada['anio_construccion'] = pd.DatetimeIndex(ciclovias_normalizada['anio_construccion']).year
ciclovias_normalizada.info()


C:\Users\ismael\AppData\Local\Temp\ipykernel_13260\3440588941.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  ciclovias_normalizada['anio_construccion'] = ciclovias_normalizada['anio_construccion'].str.replace(r'\D', '').replace('',np.nan)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 3081 non-null   int64   
 1   codigo             3081 non-null   int64   
 2   nomoficial         3081 non-null   object  
 3   alt_izqini         3081 non-null   int64   
 4   alt_izqfin         3081 non-null   int64   
 5   alt_derini         3081 non-null   int64   
 6   alt_derfin         3081 non-null   int64   
 7   nomanter           1363 non-null   object  
 8   nom_mapa           3079 non-null   object  
 9   tipo_c             3081 non-null   object  
 10  long               3081 non-null   float64 
 11  sentido            3081 non-null   object  
 12  cod_sent           3081 non-null   int64   
 13  observa            631 non-null    object  
 14  bicisenda          3081 non-null   object  
 15  lado_ciclo         950 non-null    object  
 16

Punto censales anules y mensuales

In [17]:
gdf_puntos_censos_anuales = gpd.GeoDataFrame(df_puntos_censos_anuales, geometry=gpd.points_from_xy(df_puntos_censos_anuales.X, df_puntos_censos_anuales.Y))
gdf_puntos_censos_anuales.rename(columns={'ÃREA': 'area'}, inplace=True)
gdf_puntos_censos_anuales.head(1)

,X,Y,RECORRIDO,CENTROIDE,CRUCE,area,geometry
0,-58.481712,-34.544106,R001,600.0,DEHEZA@PINTO,Barrios,POINT (-58.48171 -34.54411)


In [18]:
duplicates_df = gdf_puntos_censos_anuales['CENTROIDE']
duplicate = duplicates_df[duplicates_df.duplicated()]
duplicate

Series([], Name: CENTROIDE, dtype: float64)

In [34]:
gdf_puntos_censos_mensuales = gpd.GeoDataFrame(df_puntos_censos_mensuales, geometry=gpd.points_from_xy(df_puntos_censos_mensuales.X, df_puntos_censos_mensuales.Y))
gdf_puntos_censos_mensuales.rename(columns={'ÃREA': 'area'}, inplace=True)
gdf_puntos_censos_mensuales.drop(columns=['x_coo', 'y_coo'], inplace=True)
gdf_puntos_censos_mensuales.head(1)

,X,Y,Status,Score,PB,TIPO,CRUCE,geometry
0,-58.437811,-34.593512,M,100,PB45,ARTERIAS CON CICLOVIAS,SERRANO & LOYOLA,POINT (-58.43781 -34.59351)


Volumenes de ciclistas

In [25]:
volumen_ciclista_anual_normalizado = df_volumen_ciclista_anual
volumen_ciclista_anual_normalizado['año']= volumen_ciclista_anual_normalizado['año'].apply(str)
nan = volumen_ciclista_anual_normalizado['año'].isnull().sum()
print(nan)
volumen_ciclista_anual_normalizado.info()
volumen_ciclista_anual_normalizado.to_csv('../data bicicletas/normalizada/volumen_ciclistas_anuales_normalizada.csv', index=False, )

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9004 entries, 0 to 9003
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   centroide           9004 non-null   int64 
 1   cruce               9004 non-null   object
 2   año                 9004 non-null   object
 3   turno               9004 non-null   object
 4   cantidad_ciclistas  9004 non-null   int64 
 5   tipo_relevamiento   9004 non-null   object
dtypes: int64(2), object(4)
memory usage: 422.2+ KB


In [36]:
volumen_ciclista_mensual_normalizado = df_volumen_ciclista_mensual


volumen_ciclista_mensual_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   año                 2049 non-null   int64 
 1   mes                 2049 non-null   object
 2   punto_cruce         2049 non-null   object
 3   punto_referencia    2049 non-null   object
 4   cantidad_ciclistas  2049 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 80.2+ KB


Exportamos la normalizacion a CSVs

In [45]:
ciclovias_normalizada.to_file('../data bicicletas/normalizada/referencia_geografica_ciclovias_WGS84_normalizada.geojson', driver='GeoJSON')
gdf_puntos_censos_anuales.to_file('../data bicicletas/normalizada/puntos_censos_anuales_normalizada.geojson', driver='GeoJSON')
gdf_puntos_censos_mensuales.to_file('../data bicicletas/normalizada/puntos_censos_mensuales_normalizada.geojson', driver='GeoJSON')
volumen_ciclista_anual_normalizado.to_csv('../data bicicletas/normalizada/volumen_ciclistas_anuales_normalizada.csv', index=False)
volumen_ciclista_mensual_normalizado.to_csv('../data bicicletas/normalizada/volumen_ciclistas_mensuales_normalizada.csv', index=False)


In [28]:
df = pd.read_csv('../data bicicletas/normalizada/volumen_ciclistas_mensuales_normalizada.csv', dtype={'año': str, 'mes': str})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   año                 2049 non-null   object
 1   mes                 2049 non-null   object
 2   punto_cruce         2049 non-null   object
 3   punto_referencia    2049 non-null   object
 4   cantidad_ciclistas  2049 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 80.2+ KB
